In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_4/models/B5R2b5_5FP_1FC_fold4_1.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold4.csv') #Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(4599, 25)
(4599, 25)
Normal:  (3004, 25)
Abnormal:  (1595, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,0,0,15,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,35.0,96.0,694.0,584.0,0.036832,0.107041,0.712707,0.649402,AB01 P3-1 C015.JPG,AB01 P3-1 C015.JPG
1,1,1,1,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG,AB01 P1 C015.JPG
2,2,2,2,2,15,P2,P2,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,242.0,101.0,491.0,451.0,0.245009,0.113017,0.497278,0.501992,AB01 P2 C015.JPG,AB01 P2 C015.JPG
3,3,3,3,3,15,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,154.0,110.0,714.0,611.0,0.155235,0.122977,0.718412,0.679283,AB01 P4-2 C015.JPG,AB01 P4-2 C015.JPG
4,4,4,4,4,15,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,152.0,146.0,609.0,519.0,0.152057,0.162817,0.608229,0.577689,AB01 P4-1 C015.JPG,AB01 P4-1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 4599 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.05223133, -0.08888767, -0.19801009, ..., -0.02879767,
         0.07300981,  0.80803478],
       [-0.21011539,  0.8162728 ,  0.24005412, ..., -0.12808464,
         0.15891442, -0.05823775],
       [-0.18394434,  0.63129264,  0.11240496, ..., -0.04978415,
        -0.10629222,  0.09063857],
       ...,
       [-0.19961046,  0.1021966 ,  0.44722494, ...,  0.80394131,
         0.78595668, -0.09017114],
       [-0.2088971 , -0.03106479,  0.17585328, ...,  1.01898241,
         0.47706884, -0.06945698],
       [-0.06583431, -0.18845394, -0.2098787 , ...,  0.21689592,
        -0.22471043,  0.44389832]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.052231,-0.088888,-0.198010,-0.166597,-0.098973,-0.013442,-0.167427,0.349362,0.009032,0.553631,...,0.068065,-0.123171,0.021813,0.910158,-0.172660,0.410697,0.248255,-0.028798,0.073010,0.808035
1,-0.210115,0.816273,0.240054,0.309247,0.133062,0.097428,0.790255,0.496908,-0.124766,-0.092882,...,-0.008748,-0.198147,-0.210822,-0.124432,-0.102762,0.182280,0.409014,-0.128085,0.158914,-0.058238
2,-0.183944,0.631293,0.112405,0.339750,0.181902,0.106688,0.877962,0.156163,-0.201337,-0.131349,...,0.033220,-0.174124,-0.231143,-0.131918,0.052766,0.223142,0.424080,-0.049784,-0.106292,0.090639
3,-0.175725,-0.133661,-0.172570,-0.101319,-0.149359,0.038068,-0.213045,0.470231,-0.069968,0.603072,...,0.361357,0.169199,-0.144739,0.626173,-0.193646,-0.076549,0.081426,-0.012682,0.203793,0.696491
4,-0.103514,-0.062034,-0.144713,-0.120539,-0.014255,0.322050,-0.147671,0.640609,-0.073623,1.000547,...,0.925098,-0.162480,-0.002176,0.710623,-0.203206,0.322323,0.408586,0.029039,0.175108,1.233018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,-0.075651,-0.205475,-0.214040,-0.145242,-0.134556,-0.032651,-0.195591,0.235365,-0.211545,0.534328,...,0.976201,-0.082856,0.083209,0.501177,-0.191993,0.628379,0.776129,-0.102233,-0.103778,0.800191
4595,-0.212692,-0.158117,-0.129940,-0.167883,-0.017740,-0.004025,-0.185337,0.349680,-0.050582,0.899695,...,0.287228,-0.004580,0.112419,0.493437,-0.115886,-0.060851,0.347689,0.041572,0.423671,0.949745
4596,-0.199610,0.102197,0.447225,1.084164,-0.000168,-0.160631,0.576264,0.003632,0.623125,-0.195416,...,0.436683,0.511412,0.969955,-0.053374,0.625296,-0.169083,-0.152302,0.803941,0.785957,-0.090171
4597,-0.208897,-0.031065,0.175853,1.082890,0.077500,-0.175866,0.206401,-0.084385,0.607617,0.347168,...,-0.038317,0.453689,0.824595,0.093262,0.763576,-0.062973,-0.151034,1.018982,0.477069,-0.069457


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.052231,-0.088888,-0.198010,-0.166597,-0.098973,-0.013442,-0.167427,0.349362,0.009032,0.553631,...,-0.172660,0.410697,0.248255,-0.028798,0.073010,0.808035,Abnormal,P31,FP-B,AB01
1,-0.210115,0.816273,0.240054,0.309247,0.133062,0.097428,0.790255,0.496908,-0.124766,-0.092882,...,-0.102762,0.182280,0.409014,-0.128085,0.158914,-0.058238,Abnormal,P1,FP-A,AB01
2,-0.183944,0.631293,0.112405,0.339750,0.181902,0.106688,0.877962,0.156163,-0.201337,-0.131349,...,0.052766,0.223142,0.424080,-0.049784,-0.106292,0.090639,Abnormal,P2,FP-A,AB01
3,-0.175725,-0.133661,-0.172570,-0.101319,-0.149359,0.038068,-0.213045,0.470231,-0.069968,0.603072,...,-0.193646,-0.076549,0.081426,-0.012682,0.203793,0.696491,Abnormal,P42,FP-B,AB01
4,-0.103514,-0.062034,-0.144713,-0.120539,-0.014255,0.322050,-0.147671,0.640609,-0.073623,1.000547,...,-0.203206,0.322323,0.408586,0.029039,0.175108,1.233018,Abnormal,P41,FP-B,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,-0.075651,-0.205475,-0.214040,-0.145242,-0.134556,-0.032651,-0.195591,0.235365,-0.211545,0.534328,...,-0.191993,0.628379,0.776129,-0.102233,-0.103778,0.800191,Normal,P32,FP-B,Normal
4595,-0.212692,-0.158117,-0.129940,-0.167883,-0.017740,-0.004025,-0.185337,0.349680,-0.050582,0.899695,...,-0.115886,-0.060851,0.347689,0.041572,0.423671,0.949745,Normal,P42,FP-B,Normal
4596,-0.199610,0.102197,0.447225,1.084164,-0.000168,-0.160631,0.576264,0.003632,0.623125,-0.195416,...,0.625296,-0.169083,-0.152302,0.803941,0.785957,-0.090171,Normal,P52,FP-C,Normal
4597,-0.208897,-0.031065,0.175853,1.082890,0.077500,-0.175866,0.206401,-0.084385,0.607617,0.347168,...,0.763576,-0.062973,-0.151034,1.018982,0.477069,-0.069457,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_5FP_fold4_1.csv') #เปลี่ยนชื่อไฟล์